In [1]:
%load_ext autoreload
%autoreload 2

from pprint import pprint
from sklearn.metrics import classification_report
from utils.inference import compute_metrics


DATASET_NAME = "toxigen"
LABEL_SPACE = ["non-toxic", "toxic"]
MODEL_NAME = "deberta_large"
SEED = 42
POOLER = "mean_with_attention"
LAYER = 24

def evaluate(y_true, y_pred, is_multiclass: bool, prefix: str='test'):
    # Print some metrics
    testset_perfm = compute_metrics(
        y_true=y_true, y_pred=y_pred, is_multiclass=is_multiclass, prefix=prefix
    )
    pprint(testset_perfm)
    print(classification_report(y_true=y_true, y_pred=y_pred))


In [2]:
## Load Embeddings
from utils.io import (
    load_dataset_from_hf,
    load_labels_at_split,
    load_embeddings,
)
import numpy as np
train_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="train",
    pooler=POOLER,
    layer=LAYER
)

eval_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="eval",
    pooler=POOLER,
    layer=LAYER
)

test_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="test",
    pooler=POOLER,
    layer=LAYER
)

train_eval_embeddings = np.vstack([train_embeddings, eval_embeddings])

## Load Datasets and Labels
dataset = load_dataset_from_hf(dataset=DATASET_NAME)
train_labels = load_labels_at_split(dataset, "train")
eval_labels = load_labels_at_split(dataset, "eval")
train_eval_labels = np.concatenate([train_labels, eval_labels])
test_labels = load_labels_at_split(dataset, "test")

from datasets import DatasetDict, concatenate_datasets
train_eval_dataset = concatenate_datasets([dataset["train"], dataset["eval"]])
dataset_dict = DatasetDict(
    {"train": train_eval_dataset, "test": dataset["test"]}
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/samsoup/.cache/huggingface/token
Login successful


/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [3]:
import pickle
from sklearn.linear_model import LogisticRegression


l2 = 500
logit_clf = LogisticRegression(penalty="l2", C= 1 / l2)
logit_clf.fit(train_eval_embeddings, train_eval_labels)
predictions = logit_clf.predict(test_embeddings)


evaluate(
    y_pred=predictions, 
    y_true=test_labels, 
    is_multiclass=np.unique(test_labels).size > 2
)

# Path to save the model
model_path = 'LogisticRegression.pkl'

# Save the trained model to a .pkl file
with open(model_path, 'wb') as file:
    pickle.dump(logit_clf, file)

print(f"Model saved to {model_path}")

{'test_accuracy': 0.8202127659574469,
 'test_f1': 0.6817325800376648,
 'test_precision': 0.7327935222672065,
 'test_recall': 0.6373239436619719}
              precision    recall  f1-score   support

           0       0.85      0.90      0.87       656
           1       0.73      0.64      0.68       284

    accuracy                           0.82       940
   macro avg       0.79      0.77      0.78       940
weighted avg       0.82      0.82      0.82       940

Model saved to LogisticRegression.pkl


In [3]:
from classifiers.RandomClassifier import RandomClassifier


random_clf = RandomClassifier()
random_clf.fit(train_eval_embeddings, train_eval_labels)
predictions = random_clf.predict(test_embeddings)

evaluate(
    y_pred=predictions, 
    y_true=test_labels, 
    is_multiclass=np.unique(test_labels).size > 2
)

{'test_0_f1': 0.34888922021768304,
 'test_0_precision': 0.3504043126684636,
 'test_0_recall': 0.3473871733966746,
 'test_1_f1': 0.3348751156336725,
 'test_1_precision': 0.33241505968778695,
 'test_1_recall': 0.33737185461323393,
 'test_2_f1': 0.32285050348567,
 'test_2_precision': 0.3238036047234307,
 'test_2_recall': 0.3219029966017918,
 'test_accuracy': 0.3357084690553746,
 'test_f1': 0.3355382797790085,
 'test_macro_f1': 0.3355382797790085,
 'test_macro_precision': 0.33554099235989376,
 'test_macro_recall': 0.3355540082039001,
 'test_micro_f1': 0.3357084690553746,
 'test_micro_precision': 0.3357084690553746,
 'test_micro_recall': 0.3357084690553746,
 'test_precision': 0.33554099235989376,
 'test_recall': 0.3355540082039001,
 'test_weighted_f1': 0.3357175255192449,
 'test_weighted_precision': 0.3357449176193115,
 'test_weighted_recall': 0.3357084690553746}
              precision    recall  f1-score   support

           0       0.35      0.35      0.35      3368
           1       0